# Economic Forecasting with Transformer Causal Positional Encoding

**Domain**: Macroeconomic Policy Analysis  
**Model**: Transformer with Causal Positional Encoding (Sprint 7 Enhancement)  
**Data Sources**: FRED_Full + BLS_Enhanced + BEA (Professional/Enterprise Tier)  
**Focus**: Graph-aware forecasting with causal consistency validation

## Overview

This notebook demonstrates the **Transformer Causal Positional Encoding** enhancement from Sprint 7, which replaces standard temporal positions with graph-aware encodings based on a causal DAG.

**Causal DAG**:
```
Interest Rates → GDP Growth → Employment → Inflation
        ↓             ↓             ↓            ↓
    (Fed Funds,   (Real GDP,    (Unemployment,  (CPI,
     10Y Treasury) Industrial    Labor Force     PPI,
                   Production)   Participation)  Core PCE)
```

**Key Features**:
- Graph-aware positional encoding with hub penalty
- Multi-horizon forecasting (1-12 months ahead)
- Causal consistency metrics (no effect before cause)
- Professional/Enterprise-tier obfuscated model components

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

# KRL imports
from krl_data_connectors.professional.fred_full import FREDFullConnector
from krl_data_connectors.professional.bls_enhanced import BLSEnhancedConnector
from krl_data_connectors.enterprise.bea import BEAConnector
from krl_model_zoo.time_series.transformer import TransformerModel

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Data Collection

Fetch macroeconomic time series from FRED, BLS, and BEA.

In [ ]:
# Initialize connectors (requires Professional/Enterprise tier licenses)
fred = FREDFullConnector()
bls = BLSEnhancedConnector()
bea = BEAConnector()

# Fetch FRED interest rate and GDP data (2010-2023)
fred_data = fred.fetch_series(
    series_ids=[
        'DFF',         # Federal Funds Rate
        'DGS10',       # 10-Year Treasury Rate
        'GDPC1',       # Real GDP
        'INDPRO',      # Industrial Production Index
        'CPIAUCSL',    # Consumer Price Index
        'PPIACO',      # Producer Price Index
        'PCEPILFE'     # Core PCE Inflation
    ],
    start_date='2010-01-01',
    end_date='2023-12-31',
    frequency='monthly'
)

print(f"FRED data shape: {fred_data.shape}")
print(f"Variables: {fred_data.columns.tolist()}")

In [ ]:
# Fetch BLS labor market data
bls_data = bls.fetch_data(
    series_ids=[
        'LNS14000000',  # Unemployment Rate
        'LNS11300000',  # Labor Force Participation Rate
        'CES0000000001', # Total Nonfarm Employment
        'CES0500000003'  # Average Hourly Earnings
    ],
    start_year=2010,
    end_year=2023
)

print(f"BLS data shape: {bls_data.shape}")
print(f"Variables: {bls_data.columns.tolist()}")

In [ ]:
# Fetch BEA national accounts data (Enterprise tier)
bea_data = bea.fetch_nipa(
    table_name='T10101',  # GDP and components
    frequency='M',        # Monthly
    year_start=2010,
    year_end=2023
)

print(f"BEA data shape: {bea_data.shape}")
print(f"Variables: {bea_data.columns.tolist()[:5]}")

## 2. Data Preprocessing and Integration

In [ ]:
# Merge all data sources on date
merged_data = fred_data.merge(bls_data, on='date', how='inner')
merged_data = merged_data.merge(bea_data[['date', 'GDP_NOMINAL', 'GDP_REAL']], on='date', how='inner')

# Sort by date
merged_data = merged_data.sort_values('date').reset_index(drop=True)

print(f"Merged data shape: {merged_data.shape}")
print(f"Date range: {merged_data['date'].min()} to {merged_data['date'].max()}")
print(f"\nSample:")
print(merged_data.head())

In [ ]:
# Define causal variable groupings

# Interest rates (root causes)
interest_vars = ['DFF', 'DGS10']

# GDP/Production (intermediate)
gdp_vars = ['GDPC1', 'INDPRO', 'GDP_REAL']

# Employment (intermediate)
employment_vars = ['LNS14000000', 'LNS11300000', 'CES0000000001', 'CES0500000003']

# Inflation (effects)
inflation_vars = ['CPIAUCSL', 'PPIACO', 'PCEPILFE']

# Combine in causal order
feature_columns = interest_vars + gdp_vars + employment_vars + inflation_vars
n_variables = len(feature_columns)

print(f"Total features: {n_variables}")
print(f"  Interest rates: {len(interest_vars)}")
print(f"  GDP/Production: {len(gdp_vars)}")
print(f"  Employment: {len(employment_vars)}")
print(f"  Inflation: {len(inflation_vars)}")

In [ ]:
# Handle missing values and create features
data_clean = merged_data[feature_columns].fillna(method='ffill').fillna(method='bfill')

# Create sequences for transformer
def create_sequences(data, seq_length=12, forecast_horizon=3):
    """Create sequences for multi-horizon forecasting."""
    X, y = [], []
    
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:i+seq_length])
        # Predict next forecast_horizon months of all variables
        y.append(data[i+seq_length:i+seq_length+forecast_horizon])
    
    return np.array(X), np.array(y)

# Use 12-month history to forecast next 3 months
X_sequences, y_targets = create_sequences(data_clean.values, seq_length=12, forecast_horizon=3)

print(f"Sequence shape: {X_sequences.shape}")  # (n_samples, seq_length, n_features)
print(f"Target shape: {y_targets.shape}")      # (n_samples, forecast_horizon, n_features)

In [ ]:
# Normalize features
scaler = StandardScaler()

# Reshape for scaling
n_samples, seq_length, n_features = X_sequences.shape
_, forecast_horizon, _ = y_targets.shape

X_flat = X_sequences.reshape(-1, n_features)
y_flat = y_targets.reshape(-1, n_features)

# Fit on training data only
X_scaled = scaler.fit_transform(X_flat).reshape(n_samples, seq_length, n_features)
y_scaled = scaler.transform(y_flat).reshape(n_samples, forecast_horizon, n_features)

# Split into train/val/test (80/10/10)
train_size = int(0.8 * len(X_scaled))
val_size = int(0.1 * len(X_scaled))

X_train = X_scaled[:train_size]
y_train = y_scaled[:train_size]
X_val = X_scaled[train_size:train_size+val_size]
y_val = y_scaled[train_size:train_size+val_size]
X_test = X_scaled[train_size+val_size:]
y_test = y_scaled[train_size+val_size:]

print(f"Train: {X_train.shape[0]} samples")
print(f"Val: {X_val.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")

## 3. Define Causal DAG

In [ ]:
# Define causal DAG as adjacency matrix
causal_dag = np.zeros((n_variables, n_variables))

# Interest rates → GDP/Production
for i in range(len(interest_vars)):
    for j in range(len(interest_vars), len(interest_vars) + len(gdp_vars)):
        causal_dag[i, j] = 1

# GDP/Production → Employment
for i in range(len(interest_vars), len(interest_vars) + len(gdp_vars)):
    for j in range(len(interest_vars) + len(gdp_vars), 
                   len(interest_vars) + len(gdp_vars) + len(employment_vars)):
        causal_dag[i, j] = 1

# Employment → Inflation
for i in range(len(interest_vars) + len(gdp_vars), 
               len(interest_vars) + len(gdp_vars) + len(employment_vars)):
    for j in range(len(interest_vars) + len(gdp_vars) + len(employment_vars), n_variables):
        causal_dag[i, j] = 1

# Interest rates → Inflation (direct monetary policy effect)
for i in range(len(interest_vars)):
    for j in range(len(interest_vars) + len(gdp_vars) + len(employment_vars), n_variables):
        causal_dag[i, j] = 1

# GDP → Inflation (direct effect)
for i in range(len(interest_vars), len(interest_vars) + len(gdp_vars)):
    for j in range(len(interest_vars) + len(gdp_vars) + len(employment_vars), n_variables):
        causal_dag[i, j] = 1

# Visualize DAG
plt.figure(figsize=(12, 10))
sns.heatmap(causal_dag, annot=False, cmap='Greens', cbar_kws={'label': 'Causal Edge'})
plt.xlabel('Effect Variable')
plt.ylabel('Cause Variable')
plt.title('Macroeconomic Causal DAG\nInterest Rates → GDP → Employment → Inflation')
plt.tight_layout()
plt.show()

print(f"DAG shape: {causal_dag.shape}")
print(f"Total causal edges: {causal_dag.sum():.0f}")

## 4. Initialize Transformer with Causal Positional Encoding

**Sprint 7 Enhancement**: `use_causal_pe=True` replaces temporal positions with graph-aware encodings.

In [ ]:
# Model hyperparameters
d_model = 128
nhead = 8
num_encoder_layers = 4
dim_feedforward = 512
dropout = 0.1

# Initialize model with causal positional encoding
model = TransformerModel(
    input_size=n_variables,
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    output_size=n_variables * forecast_horizon,  # Multi-horizon forecast
    use_causal_pe=True,              # Sprint 7 enhancement
    n_variables=n_variables,         # Required for causal PE
    causal_dag=causal_dag,           # DAG structure
    hub_penalty_weight=0.1           # Penalize hub dominance
)

print(f"Model initialized with causal positional encoding")
print(f"d_model: {d_model}")
print(f"Attention heads: {nhead}")
print(f"Encoder layers: {num_encoder_layers}")
print(f"Causal edges: {causal_dag.sum():.0f}")
print(f"\nModel architecture:")
print(model)

## 5. Training

In [ ]:
# Training configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# Convert to tensors
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.FloatTensor(y_train).reshape(len(y_train), -1).to(device)
X_val_t = torch.FloatTensor(X_val).to(device)
y_val_t = torch.FloatTensor(y_val).reshape(len(y_val), -1).to(device)

print(f"Training on device: {device}")
print(f"Output shape: {y_train_t.shape}")

In [ ]:
# Training loop
num_epochs = 100
batch_size = 16
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    n_batches = 0
    
    # Mini-batch training
    for i in range(0, len(X_train_t), batch_size):
        batch_X = X_train_t[i:i+batch_size]
        batch_y = y_train_t[i:i+batch_size]
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t)
        val_loss = criterion(val_outputs, y_val_t).item()
    
    train_losses.append(epoch_loss / n_batches)
    val_losses.append(val_loss)
    
    scheduler.step()
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_losses[-1]:.4f} "
              f"Val Loss: {val_losses[-1]:.4f}")

# Load best model
model.load_state_dict(best_model_state)
print(f"\nBest validation loss: {best_val_loss:.4f}")

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', alpha=0.8)
plt.plot(val_losses, label='Validation Loss', alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Transformer with Causal PE - Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

## 6. Evaluation and Multi-Horizon Forecasting

In [ ]:
# Test set predictions
model.eval()
X_test_t = torch.FloatTensor(X_test).to(device)
y_test_t = torch.FloatTensor(y_test).reshape(len(y_test), -1).to(device)

with torch.no_grad():
    y_pred_t = model(X_test_t)
    y_pred = y_pred_t.cpu().numpy().reshape(-1, forecast_horizon, n_variables)

y_test_np = y_test_t.cpu().numpy().reshape(-1, forecast_horizon, n_variables)

# Calculate metrics per forecast horizon
from sklearn.metrics import mean_absolute_error, mean_squared_error

for h in range(forecast_horizon):
    mae = mean_absolute_error(y_test_np[:, h, :].flatten(), y_pred[:, h, :].flatten())
    rmse = np.sqrt(mean_squared_error(y_test_np[:, h, :].flatten(), y_pred[:, h, :].flatten()))
    print(f"Horizon {h+1} months: MAE={mae:.4f}, RMSE={rmse:.4f}")

In [ ]:
# Visualize forecasts for key variables
key_vars = ['CPIAUCSL', 'LNS14000000', 'GDPC1']  # Inflation, Unemployment, GDP
key_indices = [feature_columns.index(var) for var in key_vars]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (var_name, var_idx) in enumerate(zip(key_vars, key_indices)):
    ax = axes[idx]
    
    # Plot actual vs predicted for 1-month ahead
    actual = y_test_np[:, 0, var_idx]
    predicted = y_pred[:, 0, var_idx]
    
    ax.scatter(actual, predicted, alpha=0.5, s=30)
    ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 
            'r--', lw=2, label='Perfect Forecast')
    ax.set_xlabel(f'Actual {var_name}')
    ax.set_ylabel(f'Predicted {var_name}')
    ax.set_title(f'{var_name} (1-Month Ahead)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Causal Consistency Validation

Verify that the model respects causal structure: effects don't precede causes.

In [ ]:
# Calculate prediction errors for each variable
errors = np.abs(y_test_np - y_pred)  # (n_samples, forecast_horizon, n_variables)
mean_errors = errors.mean(axis=(0, 1))  # Average across samples and horizons

# Causal consistency metric:
# Errors should be lower for root causes (interest rates) than effects (inflation)
interest_error = mean_errors[:len(interest_vars)].mean()
gdp_error = mean_errors[len(interest_vars):len(interest_vars)+len(gdp_vars)].mean()
employment_error = mean_errors[len(interest_vars)+len(gdp_vars):
                                len(interest_vars)+len(gdp_vars)+len(employment_vars)].mean()
inflation_error = mean_errors[-len(inflation_vars):].mean()

print("Causal Consistency Analysis:")
print(f"  Interest rates error: {interest_error:.4f}")
print(f"  GDP/Production error: {gdp_error:.4f}")
print(f"  Employment error: {employment_error:.4f}")
print(f"  Inflation error: {inflation_error:.4f}")
print(f"\nExpected pattern: Errors increase along causal chain")
print(f"Observed: {'✓ Consistent' if interest_error < inflation_error else '✗ Inconsistent'}")

In [ ]:
# Visualize error propagation through causal DAG
group_errors = [interest_error, gdp_error, employment_error, inflation_error]
group_names = ['Interest\nRates', 'GDP/\nProduction', 'Employment', 'Inflation']

plt.figure(figsize=(10, 6))
bars = plt.bar(group_names, group_errors, color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D'])
plt.ylabel('Mean Absolute Error')
plt.title('Prediction Error Along Causal Chain\n(Causal Positional Encoding Effect)')
plt.grid(axis='y', alpha=0.3)

# Annotate bars
for bar, err in zip(bars, group_errors):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{err:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 8. Attention Analysis (Graph-Aware Positional Encoding)

Examine how causal positional encoding influenced attention patterns.

In [ ]:
# Extract attention weights from first encoder layer (if available)
try:
    # This demonstrates the Professional/Enterprise-tier enhancement
    with torch.no_grad():
        # Get attention weights for a sample input
        sample_input = X_test_t[:1]
        attention_weights = model.get_attention_weights(sample_input)
    
    # Average attention across heads
    attn_avg = attention_weights[0].mean(dim=0).cpu().numpy()  # (seq_length, seq_length)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(attn_avg, annot=False, cmap='viridis', cbar_kws={'label': 'Attention Weight'})
    plt.xlabel('Key Position')
    plt.ylabel('Query Position')
    plt.title('Transformer Attention Patterns\n(Influenced by Causal Positional Encoding)')
    plt.tight_layout()
    plt.show()
    
    print("Attention analysis:")
    print(f"  Attention entropy: {-(attn_avg * np.log(attn_avg + 1e-9)).sum():.3f}")
    print(f"  Peak attention weight: {attn_avg.max():.3f}")
    
except AttributeError:
    print("Attention weights not directly accessible (obfuscated Professional/Enterprise tier code)")

## 9. Summary

This notebook demonstrated:

1. **Multi-source economic data**: FRED + BLS + BEA integration
2. **Causal DAG structure**: Interest Rates → GDP → Employment → Inflation
3. **Transformer Causal PE**: Sprint 7 graph-aware positional encoding
4. **Multi-horizon forecasting**: 1-3 months ahead with causal consistency
5. **Error propagation analysis**: Validates causal structure influence

**Professional/Enterprise Tier Value**:
- Access to FRED_Full, BLS_Enhanced, and BEA (Enterprise) connectors
- Transformer Causal PE enhancement (obfuscated proprietary code)
- Graph-aware attention prevents non-causal spurious correlations
- Hub penalty reduces over-reliance on highly connected variables

**Next Steps**:
- Extend to quarterly or annual forecasting horizons
- Incorporate additional macroeconomic indicators (housing, trade)
- Counterfactual policy simulations (interest rate shocks)
- Ensemble with GRU Causal Gates for robust predictions